# Keras MNIST Model Deployment

 * Wrap a Tensorflow MNIST python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install seldon-core
pip install keras
```

## Train locally
 

In [1]:
import numpy as np
import math
import datetime
#from seldon.pipeline import PipelineSaver
import os
import tensorflow as tf
from keras import backend
from keras.models import Model,load_model
from keras.layers import Dense,Input
from keras.layers import Dropout
from keras.layers import Flatten, Reshape
from keras.constraints import maxnorm
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D

from keras.callbacks import TensorBoard

class MnistFfnn(object):

    def __init__(self,
                 input_shape=(784,),
                 nb_labels=10,
                 optimizer='Adam',
                 run_dir='tensorboardlogs_test'):
        
        self.model_name='MnistFfnn'
        self.run_dir=run_dir
        self.input_shape=input_shape
        self.nb_labels=nb_labels
        self.optimizer=optimizer
        self.build_graph()

    def build_graph(self):
                            
        inp = Input(shape=self.input_shape,name='input_part')

        #keras layers
        with tf.name_scope('dense_1') as scope:
            h1 = Dense(256,
                         activation='relu',
                         W_constraint=maxnorm(3))(inp)
            drop1 = Dropout(0.2)(h1)

        with tf.name_scope('dense_2') as scope:
            h2 = Dense(128,
                       activation='relu',
                       W_constraint=maxnorm(3))(drop1)
            drop2 = Dropout(0.5)(h2)
            
            out = Dense(self.nb_labels,
                        activation='softmax')(drop2)

        self.model = Model(inp,out)
        
        if self.optimizer ==  'rmsprop':
            self.model.compile(loss='categorical_crossentropy',
                               optimizer='rmsprop',
                               metrics=['accuracy'])
        elif self.optimizer == 'Adam':
            self.model.compile(loss='categorical_crossentropy',
                               optimizer='Adam',
                               metrics=['accuracy'])
            
        print('graph builded')

    def fit(self,X,y=None,
            X_test=None,y_test=None,
            batch_size=128,
            nb_epochs=2,
            shuffle=True):
        
        now = datetime.datetime.now()
        tensorboard_logname = self.run_dir+'/{}_{}'.format(self.model_name,
                                                           now.strftime('%Y.%m.%d_%H.%M'))      
        tensorboard = TensorBoard(log_dir=tensorboard_logname)
        
        self.model.fit(X,y,
                       validation_data=(X_test,y_test),
                       callbacks=[tensorboard],
                       batch_size=batch_size, 
                       nb_epoch=nb_epochs,
                       shuffle = shuffle)
        return self
    
    def predict_proba(self,X):

        return self.model.predict_proba(X)
    
    def predict(self, X):
        probas = self.model.predict_proba(X)
        return([[p>0.5 for p in p1] for p1 in probas])
        
    def score(self, X, y=None):
        pass

    def get_class_id_map(self):
        return ["proba"]

class MnistConv(object):

    def __init__(self,
                 input_shape=(784,),
                 nb_labels=10,
                 optimizer='Adam',
                 run_dir='tensorboardlogs_test',
                 saved_model_file='MnistClassifier.h5'):
        
        self.model_name='MnistConv'
        self.run_dir=run_dir
        self.input_shape=input_shape
        self.nb_labels=nb_labels
        self.optimizer=optimizer
        self.saved_model_file=saved_model_file
        self.build_graph()

    def build_graph(self):
                                                                
        inp = Input(shape=self.input_shape,name='input_part')
        inp2 = Reshape((28,28,1))(inp)      
        #keras layers
        with tf.name_scope('conv') as scope:
            conv = Convolution2D(32, 3, 3,
                                 input_shape=(32, 32, 3),
                                 border_mode='same',
                                 activation='relu',
                                 W_constraint=maxnorm(3))(inp2)
            drop_conv = Dropout(0.2)(conv)
            max_pool = MaxPooling2D(pool_size=(2, 2))(drop_conv)

        with tf.name_scope('dense') as scope:
            flat = Flatten()(max_pool)                
            dense = Dense(128,
                          activation='relu',
                          W_constraint=maxnorm(3))(flat)
            drop_dense = Dropout(0.5)(dense)
            
            out = Dense(self.nb_labels,
                        activation='softmax')(drop_dense)

        self.model = Model(inp,out)
        
        if self.optimizer ==  'rmsprop':
            self.model.compile(loss='categorical_crossentropy',
                               optimizer='rmsprop',
                               metrics=['accuracy'])
        elif self.optimizer == 'Adam':
            self.model.compile(loss='categorical_crossentropy',
                               optimizer='Adam',
                               metrics=['accuracy'])
            
        print('graph builded')

    def fit(self,X,y=None,
            X_test=None,y_test=None,
            batch_size=128,
            nb_epochs=2,
            shuffle=True):
        
        now = datetime.datetime.now()
        tensorboard_logname = self.run_dir+'/{}_{}'.format(self.model_name,
                                                           now.strftime('%Y.%m.%d_%H.%M'))      
        tensorboard = TensorBoard(log_dir=tensorboard_logname)
        
        self.model.fit(X,y,
                       validation_data=(X_test,y_test),
                       callbacks=[tensorboard],
                       batch_size=batch_size, 
                       nb_epoch=nb_epochs,
                       shuffle = shuffle)
        #if not os.path.exists('saved_model'):
        #    os.makedirs('saved_model')
        self.model.save(self.saved_model_file)
        return self
    
    def predict_proba(self,X):
        return self.model.predict_proba(X)
    
    def predict(self, X):
        probas = self.model.predict_proba(X)
        return([[p>0.5 for p in p1] for p1 in probas])
        
    def score(self, X, y=None):
        pass

    def get_class_id_map(self):
        return ["proba"]



Using TensorFlow backend.


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/MNIST_data', one_hot=True)
X_train = mnist.train.images
y_train = mnist.train.labels
X_test = mnist.test.images
y_test = mnist.test.labels
mc = MnistConv()
mc.fit(X_train,y=y_train,
    X_test=X_test,y_test=y_test)



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


/home/clive/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:126: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu", padding="same", kernel_constraint=<keras.con...)`
/home/clive/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:134: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="relu", kernel_constraint=<keras.con...)`
/home/clive/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:169: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


graph builded
Train on 55000 samples, validate on 10000 samples
Epoch 1/2
55000/55000 [==============================] - 25s 463us/step - loss: 0.3302 - acc: 0.9025 - val_loss: 0.1015 - val_acc: 0.9727
Epoch 2/2
55000/55000 [==============================] - 27s 488us/step - loss: 0.1227 - acc: 0.9642 - val_loss: 0.0633 - val_acc: 0.9798


Wrap model using s2i

In [3]:
!s2i build . seldonio/seldon-core-s2i-python3:0.18 keras-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Build completed successfully


In [4]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 keras-mnist:0.1

6231efde4036974469fddd42585db66067e6a30bcfd40efe5cf474d385f0eeda


Send some random features that conform to the contract

In [5]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[0.387 0.103 0.152 0.129 0.211 0.088 0.659 0.028 0.663 0.666 0.134 0.396
  0.704 0.089 0.407 0.896 0.734 0.375 0.109 0.796 0.917 0.186 0.736 0.013
  0.565 0.256 0.405 0.205 0.317 0.342 0.02  0.748 0.496 0.376 0.405 0.712
  0.775 0.904 0.277 0.973 0.004 0.996 0.692 0.802 0.967 0.361 0.222 0.358
  0.73  0.032 0.516 0.945 0.734 0.012 0.807 0.558 0.604 0.978 0.111 0.772
  0.276 0.484 0.645 0.73  0.953 0.306 0.049 0.299 0.872 0.197 0.389 0.191
  0.604 0.431 0.498 0.091 0.366 0.834 0.266 0.256 0.827 0.996 0.071 0.522
  0.108 0.063 0.607 0.126 0.97  0.758 0.99  0.961 0.285 0.547 0.633 0.788
  0.619 0.694 0.157 0.91  0.992 0.276 0.422 0.978 0.108 0.272 0.605 0.375
  0.964 0.257 0.215 0.583 0.594 0.162 0.118 0.518 0.026 0.687 0.98  0.666
  0.233 0.998 0.678 0.379 0.778 0.149 0.889 0.911 0.019 0.183 0.471 0.272
  0.513 0.628 0.769 0.062 0.706 0.029 0.31  0.322 0.341 0.492 0.124 0.154
  0.643 0.145 0.966 0.874 0.364 0.009 0.611 0.073

In [6]:
!docker rm mnist_predictor --force

mnist_predictor


# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [7]:
!minikube start --memory 4096 

😄  minikube v1.0.0 on linux (amd64)
🤹  Downloading Kubernetes v1.14.0 images in the background ...
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
🐳  Version of container runtime is 18.06.2-ce
⌛  Waiting for image downloads to complete ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.14.0 ...
🚀  Launching Kubernetes v1.14.0 using kubeadm ... 
⌛  Waiting for pods: apiserver proxy etcd scheduler controller dns
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](../../seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

In [17]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.18 keras-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Build completed successfully


In [18]:
!kubectl create -f keras_mnist_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [19]:
!kubectl rollout status deploy/keras-mnist-deployment-keras-mnist-predictor-8baf5cc

Waiting for deployment "keras-mnist-deployment-keras-mnist-predictor-8baf5cc" rollout to finish: 0 of 1 updated replicas are available...
deployment "keras-mnist-deployment-keras-mnist-predictor-8baf5cc" successfully rolled out


In [20]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    seldon-deployment-example --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[0.615 0.937 0.603 0.929 0.9   0.267 0.498 0.514 0.13  0.579 0.213 0.063
  0.671 0.524 0.455 0.049 0.159 0.379 0.886 0.302 0.024 0.57  0.86  0.979
  0.908 0.502 0.427 0.818 0.711 0.83  0.496 0.908 0.567 0.065 0.639 0.464
  0.699 0.415 0.356 0.181 0.152 0.409 0.901 0.981 0.648 0.761 0.721 0.867
  0.76  0.834 0.092 0.236 0.881 0.292 0.229 0.37  0.069 0.413 0.007 0.15
  0.132 0.851 0.75  0.026 0.614 0.533 0.082 0.805 0.176 0.662 0.379 0.002
  0.001 0.132 0.345 0.016 0.317 0.418 0.197 0.846 0.956 0.193 0.447 0.835
  0.2   0.313 0.094 0.94  0.068 0.724 0.732 0.561 0.763 0.589 0.056 0.893
  0.867 0.548 0.365 0.865 0.459 0.217 0.686 0.831 0.952 0.526 0.567 0.544
  0.84  0.642 0.659 0.266 0.666 0.401 0.77  0.646 0.477 0.646 0.186 0.39
  0.197 0.216 0.552 0.465 0.294 0.596 0.955 0.117 0.644 0.31  0.925 0.559
  0.113 0.897 0.379 0.307 0.581 0.044 0.644 0.31  0.871 0.001 0.266 0.356
  0.17  0.16  0.761 0.035 0.217 0.417 0.877 0.862 0

In [ ]:
!minikube delete